In [5]:
## Imports

# utility modules
import glob
import os
import sys
import re

# the usual suspects:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# specialty modules
import h5py
import pyproj
from scipy.interpolate import interp1d


sys.path.append("/home/jovyan/surface_velocity/scripts")
from loading_scripts import atl06_to_dict

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#! cd ..; [ -d pointCollection ] || git clone https://www.github.com/smithB/pointCollection.git
sys.path.append(os.path.join(os.getcwd(), '..'))
import pointCollection as pc

In [7]:
from readers.read_HDF5_ATL03 import read_HDF5_ATL03
from readers.get_ATL03_x_atc import get_ATL03_x_atc
from readers.ATL06_to_dict import ATL06_to_dict

In [8]:
# data in Pangeo
data='/srv/shared/surface_velocity/'
data_root='/srv/tutorial-data/land_ice_applications/'

In [9]:
#spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-59.2, -81.6, -55.16, -81])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
MOA=pc.grid.data().from_geotif(os.path.join(data_root, 'MOA','moa_2009_1km.tif'), bounds=[XR, YR])

# show the mosaic:
plt.figure()
MOA.show(cmap='gray', clim=[14000, 17000])
plt.plot(xy[0,:], xy[1,:])
plt.title('Mosaic of Antarctica for FIS')

[-81.6 -81.  -81.  -81.6 -81.6]
[-55.16 -55.16 -59.2  -59.2  -55.16]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([-840950., -750950.,  468825.,  558825.]), 'origin': 'lower'}


Text(0.5, 1.0, 'Mosaic of Antarctica for FIS')

In [11]:
## Get data into dictionary

data_root='/srv/shared/surface_velocity/'
#ATL06_files=glob.glob(os.path.join(data_root, 'FIS_ATL06_small', '*.h5'))
ATL06_files=glob.glob(os.path.join(data_root, 'FIS_ATL06', '*.h5'))

D_dict={}
error_count=0
for file in ATL06_files:
    try:
        D_dict[file]=atl06_to_dict(file, '/gt2l', index=slice(0, -1, 25), epsg=3031)
    except KeyError as e:
        print(f'file {file} encountered error {e}')
        error_count += 1
print(f"read {len(D_dict)} data files of which {error_count} gave errors")

file /srv/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190430122344_04920311_003_01.h5 encountered error 'Unable to open object (component not found)'
file /srv/shared/surface_velocity/FIS_ATL06/processed_ATL06_20181030210407_04920111_003_01.h5 encountered error 'Unable to open object (component not found)'
file /srv/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190730080323_04920411_003_01.h5 encountered error 'Unable to open object (component not found)'
file /srv/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190220230230_08320211_003_01.h5 encountered error 'Unable to open object (component not found)'
file /srv/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190312235510_11380211_003_01.h5 encountered error 'Unable to open object (component not found)'
file /srv/shared/surface_velocity/FIS_ATL06/processed_ATL06_20181108184743_06280111_003_01.h5 encountered error 'Unable to open object (component not found)'
file /srv/shared/surface_velocity/FIS_ATL06/processe

In [12]:
D_2l={}
D_2r={}

# specify the rgt here:
rgt="0848"
# iterate over the repeat cycles
for cycle in ['03','04','05','06','07']:
    for filename in glob.glob(os.path.join(data, 'FIS_ATL06', f'*ATL06_*_{rgt}{cycle}*_003*.h5')):
        try:
            # read the left-beam data
            D_2l[filename]=atl06_to_dict(filename,'/gt3l', index=None, epsg=3031)
            # read the right-beam data
            D_2r[filename]=atl06_to_dict(filename,'/gt3r', index=None, epsg=3031)
            # plot the locations in the previous plot
            map_ax.plot(D_2r[filename]['x'], D_2r[filename]['y'],'k');  
            map_ax.plot(D_2l[filename]['x'], D_2l[filename]['y'],'k');
        except Exception as e:
            print(f'filename={filename}, exception={e}')
            
plt.figure();
for filename,  Di in D_2r.items():
    #Plot only points that have ATL06_quality_summary==0 (good points)
    #hl=plot_segs(Di, ind=Di['atl06_quality_summary']==0, label=f"cycle={Di['cycle']}")
    hl=plt.plot(Di['x_atc'][Di['atl06_quality_summary']==0], Di['h_li'][Di['atl06_quality_summary']==0], '.', label=f"cycle={Di['cycle']}")
    
plt.legend()
plt.xlabel('x_atc')
plt.ylabel('elevation');


filename=/srv/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190523195046_08480311_003_01.h5, exception=name 'map_ax' is not defined
filename=/srv/shared/surface_velocity/FIS_ATL06/processed_ATL06_20190822153035_08480411_003_01.h5, exception=name 'map_ax' is not defined


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
plt.figure();
for filename,  Di in D_2r.items():
    #Plot only points that have ATL06_quality_summary==0 (good points)
    #hl=plot_segs(Di, ind=Di['atl06_quality_summary']==0, label=f"cycle={Di['cycle']}")
    hl=plt.plot(Di['x_atc'], Di['h_li'], '.', label=f"cycle={Di['cycle']}")
    
plt.legend()
plt.xlabel('x_atc')
plt.ylabel('elevation');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
D_2l={}
D_2r={}

# specify the rgt here:
rgt="0848"
# iterate over the repeat cycles
for cycle in ['03','04','05','06','07']:
    for filename in glob.glob(os.path.join(data, 'FIS_ATL06', f'*ATL06_*_{rgt}{cycle}*_003*.h5')):
        try:
            # read the left-beam data
            D_2l[filename]=atl06_to_dict(filename,'/gt3l', index=None, epsg=3031)
            # read the right-beam data
            D_2r[filename]=atl06_to_dict(filename,'/gt3r', index=None, epsg=3031)
            
        except Exception as e:
            print(f'filename={filename}, exception={e}')
df_is2 = []
for filename, Di in D_2r.items():
    #Plot only points that have ATL06_quality_summary==0 (good points)
    #hl=plot_segs(Di, ind=Di['atl06_quality_summary']==0, label=f"cycle={Di['cycle']}")
    df_is2.append(pd.DataFrame.from_dict(Di))
    


In [15]:
### correlation
df = pd.merge(df_is2[0], df_is2[1], how='inner', on='x_atc')

is2_cor = np.corrcoef(df.h_li_x, df.h_li_y)
